In [3]:
""""import cv2 as cv
import numpy as np

In [4]:
""""cap = cv.VideoCapture(r"C:\Users\18110\Downloads\volleyball_match.mp4")

ret, frame1 = cap.read()
ret, frame2 = cap.read()

while True:
    diff = cv.absdiff(frame1 , frame2)
    gray = cv.cvtColor(diff , cv.COLOR_BGR2GRAY)
    blur = cv.GaussianBlur(gray , (5,5) , 0)##
    _, thresh = cv.threshold(blur , 20 , 255, cv.THRESH_BINARY)
    dilated = cv.dilate(thresh, None, iterations = 3)##
    contours, _ = cv.findContours(dilated, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

    hsv = cv.cvtColor(diff , cv.COLOR_BGR2HSV)

   # lower_color = np.array([15 , 170 , 170])
    #upper_color = np.array([25 , 255 , 255])
    #mask = cv.inRange(hsv , lower_color , upper_color)
    #kernal = np.ones((5,5) , np.uint8)

    
    #mask = cv.morphologyEx(mask , cv.MORPH_OPEN , kernal)

    #ball_contours, _ = cv.findContours(mask, cv.RETR_TREE , cv.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        (x , y, w, h) = cv.boundingRect(contour)
        if cv.contourArea(contour) < 700:
            continue
        cv.rectangle(frame1, (x , y) , (x+w , y+h) , (0,255,0) , 2)
        cv.putText(frame1 , "Players", (10,20), cv.FONT_HERSHEY_SIMPLEX , 1, (0,0,255), 3)
    #cv.drawContours(frame1, contours, -1, (0,255,0) , 2)

    cv.imshow('Video' , frame1)
    frame1 = frame2
    ret, frame2 = cap.read()

    
    if cv.waitKey(25)  & 0xFF == ord('q'):
        break
cap.release()
cv.destroyAllWindows()


In [22]:
"video = cv.VideoCapture(r"C:\Users\18110\Downloads\volleyball_match.mp4")
video.get(cv.CAP_PROP_FPS)
#print(int(1000/28.99))

28.999522097452342

HSV at (887,45): [171 255  94]
HSV at (760,205): [160  63  97]
HSV at (344,177): [174  55 243]
HSV at (664,70): [173 189  35]
HSV at (1019,173): [168 194 104]
HSV at (901,168): [170 205 128]
HSV at (771,115): [ 20 158 255]
HSV at (847,283): [ 83 100 133]
HSV at (692,109): [175 121 131]
HSV at (256,481): [  4 198 198]
HSV at (920,288): [ 90  98 125]
HSV at (978,531): [  0 132 221]
HSV at (978,531): [  0 133 224]
HSV at (769,690): [ 13 207 200]
HSV at (1050,131): [  8 177 127]
HSV at (477,129): [ 19 206 224]
HSV at (477,129): [ 20 208 222]
HSV at (477,129): [ 20 208 222]


In [1]:
import cv2
import numpy as np

cap = cv2.VideoCapture("C:\\Users\\18110\\Downloads\\volleyball_match.mp4")

# Background subtractor
fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=True)

# Kalman filter
kalman = cv2.KalmanFilter(4, 2)
kalman.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
kalman.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)
kalman.processNoiseCov = np.eye(4, dtype=np.float32) * 0.03

while True:
    ret, frame = cap.read()
    if not ret:
        break

    #background subtraction*
    fgmask = fgbg.apply(frame)

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_yellow = np.array([15, 80, 80])  
    upper_yellow = np.array([40, 255, 255])
    mask = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # Mask & yellow detectiob
    mask = cv2.bitwise_and(mask, fgmask)

    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    best_ball = None
    for contour in contours:
        # Circular objects 
        area = cv2.contourArea(contour)
        if area > 50:  
            (x, y), radius = cv2.minEnclosingCircle(contour)
            circularity = (4 * np.pi * area) / (cv2.arcLength(contour, True) ** 2)
            if 0.6 < circularity < 1.2:  
                best_ball = (int(x), int(y), int(radius))
                break

    if best_ball:
        x, y, radius = best_ball
        measured = np.array([[np.float32(x)], [np.float32(y)]])
        kalman.correct(measured)

    # Predict ball position
    predicted = kalman.predict()
    x, y = int(predicted[0]), int(predicted[1])


    cv2.circle(frame, (x, y), 10, (0, 0, 255), -1)
    cv2.putText(frame, "Predicted Ball", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    cv2.imshow("Kalman Tracking", frame)
    cv2.imshow("Ball Detection", mask)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

C:\Users\18110\AppData\Local\Temp\ipykernel_21032\969197733.py:53: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x, y = int(predicted[0]), int(predicted[1])


C:\Users\18110\AppData\Local\Temp\ipykernel_16984\2679043229.py:69: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x, y = int(predicted[0]), int(predicted[1])
